In [1]:
import numpy as np
from math import radians, cos, sin, asin, sqrt

BASE_PRICE = 10
MAX_PRICE = 2 * BASE_PRICE
MIN_PRICE = 0.5 * BASE_PRICE

In [22]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return R * c

def model1_linear(prev_price, occupancy, capacity, alpha=2):
    ratio = occupancy / capacity
    new_price = prev_price + alpha * ratio
    return np.clip(new_price, MIN_PRICE, MAX_PRICE)

def compute_demand(occupancy, capacity, queue, traffic, is_special,
                   alpha=1, beta=0.5, gamma=0.7, delta=1.2):
    demand = (
        alpha * (occupancy / capacity) +
        beta * queue -
        gamma * traffic +
        delta * is_special
    )

    traffic_mapping = {'low': 1, 'average': 2, 'high': 3}
    traffic_numeric = traffic_mapping.get(traffic, 1)
    demand = (
        alpha * (occupancy / capacity) +
        beta * queue -
        gamma * traffic_numeric +
        delta * is_special
    )

    return demand

In [3]:
def normalize_demand(demand, min_d=0, max_d=10):
    return np.clip((demand - min_d) / (max_d - min_d), 0, 1)

def model2_demand_based(demand):
    lambda_coeff = 1
    price = BASE_PRICE * (1 + lambda_coeff * normalize_demand(demand))
    return np.clip(price, MIN_PRICE, MAX_PRICE)

def model3_competitive(my_price, my_lat, my_lon, lot_id, current_df):
    nearby_prices = []
    for _, row in current_df.iterrows():
        if row['SystemCodeNumber'] == lot_id:
            continue
        dist = haversine(my_lat, row['Longitude'], row['Latitude'], row['Longitude'])
        if dist < 1.0:
            nearby_prices.append(row['Price'])
    avg_nearby = np.mean(nearby_prices) if nearby_prices else my_price
    if my_price > avg_nearby:
        return my_price - 1
    elif my_price < avg_nearby:
        return my_price + 1
    return my_price


In [28]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot


def plot_dashboard(sim_df):
    sample_lots = sim_df['SystemCodeNumber'].unique()[:4]
    plots = []

    for lot in sample_lots:
        data = sim_df[sim_df['SystemCodeNumber'] == lot]
        src1 = ColumnDataSource(data={"x": data['Timestamp'], "y": data['Price']})
        src2 = ColumnDataSource(data={"x": data['Timestamp'], "y": data['Occupancy']})

        p1 = figure(title=f"Price Over Time - Lot {lot}", x_axis_type='datetime', width=450, height=250)
        p1.line("x", "y", source=src1, line_width=2, color="navy")

        p2 = figure(title=f"Occupancy Over Time - Lot {lot}", x_axis_type='datetime', width=450, height=250)
        p2.line("x", "y", source=src2, line_width=2, color="firebrick")

        plots.append([p1, p2])

    show(gridplot(plots))

In [29]:
def simulate_prices(df):
  df = df.sort_values('Timestamp')
  price_dict = {}
  results = []

  traffic_mapping = {'low': 1, 'average': 2, 'high': 3}
  df['TrafficConditionNearby_numeric'] = df['TrafficConditionNearby'].map(traffic_mapping).fillna(1) # Default to low if not found


  for t in df['Timestamp'].unique():
      current = df[df['Timestamp'] == t].copy()
      prices = []
      for idx, row in current.iterrows():
          prev_price = price_dict.get(row['SystemCodeNumber'], BASE_PRICE)
          demand = compute_demand(
              row['Occupancy'], row['Capacity'], row['QueueLength'],
              row['TrafficConditionNearby_numeric'], row['IsSpecialDay']
          )
          new_price = model2_demand_based(demand)
          current.at[idx, 'Price'] = new_price
          new_price = model3_competitive(new_price, row['Latitude'], row['Longitude'], row['SystemCodeNumber'], current)
          price_dict[row['SystemCodeNumber']] = new_price
          prices.append(new_price)
      current['Price'] = prices
      results.append(current)
  return pd.concat(results)

In [31]:
import pandas as pd

try:
    df = pd.read_csv('/dataset.csv')

    df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
except FileNotFoundError:
    print("Error: '/dataset.csv' not found. Please replace '/dataset.csv' with the correct file path.")
    df = None
if df is not None:

    simulated_df = simulate_prices(df.copy())


    plot_dashboard(simulated_df)

In [40]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot

def plot_enhanced_dashboard(sim_df):
    sample_lots = sim_df['SystemCodeNumber'].unique()[:4]
    plots = []

    for lot in sample_lots:
        data = sim_df[sim_df['SystemCodeNumber'] == lot]
        src1 = ColumnDataSource(data={"x": data['Timestamp'], "y": data['Price']})
        src2 = ColumnDataSource(data={"x": data['Timestamp'], "y": data['Occupancy']})

        p1 = figure(title=f"Price Over Time - Lot {lot}", x_axis_type='datetime', width=450, height=250)
        p1.line("x", "y", source=src1, line_width=2, color="navy")

        p2 = figure(title=f"Occupancy Over Time - Lot {lot}", x_axis_type='datetime', width=450, height=250)
        p2.line("x", "y", source=src2, line_width=2, color="firebrick")

        plots.append([p1, p2])

    show(gridplot(plots))

In [44]:
simulated_df.to_csv("./simulated_parking_prices.csv", index=False)
print("\n✅ Dynamic pricing simulation complete.")
print("📁 CSV exported: simulated_parking_prices.csv")
print("📊 Dashboard visualized with Bokeh.")


✅ Dynamic pricing simulation complete.
📁 CSV exported: simulated_parking_prices.csv
📊 Dashboard visualized with Bokeh.


In [46]:
from bokeh.io.export import export_png
from bokeh.resources import INLINE
from bokeh.embed import file_html
import os

In [57]:
def export_dashboard(sim_df):
  from bokeh.plotting import figure
  from bokeh.layouts import gridplot
  from bokeh.models import ColumnDataSource
  from bokeh.resources import INLINE
  from bokeh.embed import file_html
  from bokeh.io.export import export_png
  import os

  plots = []
  lots = sim_df['SystemCodeNumber'].unique()[:4]
  for lot in lots:
      data = sim_df[sim_df['SystemCodeNumber'] == lot]
      src1 = ColumnDataSource(data={"x": data['Timestamp'], "y": data['Price']})

      p = figure(title=f"Price Over Time - Lot {lot}", x_axis_type='datetime', width=600, height=300)
      p.line("x", "y", source=src1, line_width=2, color="green")
      plots.append(p)

  dashboard = gridplot(plots, ncols=1)
  html = file_html(dashboard, INLINE, "Dynamic Pricing Dashboard")
  with open("./final_dashboard.html", "w") as f:
      f.write(html)
  print("✅ Exported HTML: final_dashboard.html")

  # Optional: Export PNG for first plot only (others can be added similarly)
  try:
      export_png(plots[0], filename="./price_plot_lot1.png")
      print("✅ Exported PNG: price_plot_lot1.png")
  except Exception as e:
      print("⚠️ PNG export failed:", e)

In [64]:
export_dashboard(simulated_df)
simulated_df.to_csv("./simulated_parking_prices.csv", index=False)
print("\n✅ Dynamic pricing simulation complete.")
print("📁 CSV exported: simulated_parking_prices.csv")
print("📊 Dashboard visualized and exported as HTML/PNG.")

✅ Exported HTML: final_dashboard.html
⚠️ PNG export failed: To use bokeh.io image export functions you need selenium ('conda install selenium' or 'pip install selenium')

✅ Dynamic pricing simulation complete.
📁 CSV exported: simulated_parking_prices.csv
📊 Dashboard visualized and exported as HTML/PNG.
